In [ ]:
####### XGBoost Forecasting #######

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
from xgboost import XGBRegressor
import lightgbm as lgbm

matplotlib.rcParams['figure.figsize'] = (12, 8)
matplotlib.rcParams['axes.grid'] = False

In [ ]:
# Loss function
def mape(y_pred, y_train):
    ape = np.abs((y_pred - y_train) / y_train)
    ape[~np.isfinite(ape)] = 1
    return np.mean(ape)

In [ ]:
df = pd.read_excel('Train_data.xlsx')
df[datetime(2020, 4, 1, 0, 0)] = (df[datetime(2020, 3, 1, 0, 0)] + df[datetime(2020, 5, 1, 0, 0)])/2
df = df.melt(id_vars=['Warehouse id', 'Region', 'SKU id'], var_name='date', value_name='sales')
df['SKU id'] = df['SKU id'].str.extract('(\d+)',expand = False).astype(int)
df.rename(columns={'SKU id':'store_id'}, inplace=True)
df = pd.concat([df,pd.get_dummies(df['Region'])], axis = 1)
df.drop('Region', axis = 1, inplace = True)
df['Year'] = df['date'].apply(lambda time: time.year)
df['Month'] = df['date'].apply(lambda time: time.month)
df.sort_values(['Warehouse id','store_id']).drop('Warehouse id', axis = 1)
df['isFest'] = df['date'].apply(lambda x: x.month == 3 or x.month == 5 or x.month == 11 or x.month == 12).astype(int)
df['isSum'] =  df['date'].apply(lambda x: x.month >= 4 and x.month<=7).astype(int)
df['isWin'] =  df['date'].apply(lambda x: x.month == 10 or x.month == 11 or x.month == 12 or x.month == 1 or x.month == 2).astype(int)
df = df.sort_values(['Warehouse id','store_id'])#.drop('Warehouse id', axis = 1)
df['next'] = df.groupby(['store_id', 'Warehouse id'])['sales'].shift(-1)
df['next'] = df['next'].fillna(df['next'].median())
# df['next'].fillna(df.groupby(['store_id', 'Warehouse id'])['sales'].median())
# df.head()
a = df.groupby(['store_id', 'Warehouse id'])['sales'].rolling(6).mean()
df['ra'] = a.fillna(a.mean()).reset_index(level=0, drop=True).reset_index(level=0, drop=True)
df['ewm'] = df.groupby(['store_id', 'Warehouse id'])['sales'].ewm(alpha = 0.2).mean().reset_index(level=0, drop=True).reset_index(level = 0, drop = True)
df.fillna(df.median(), inplace = True)

split_pt = '2020-10-01'
train_data = df[df['date'] <= split_pt]
test_data = df[df['date'] > split_pt]
# train_data

train_data.drop(['Warehouse id', 'date', 'store_id'], axis = 1, inplace = True)
test_data.drop(['Warehouse id', 'date', 'store_id'], axis = 1, inplace = True)

X = train_data.drop(['next', 'Month', 'Year'], axis = 1)
# X = train_data.drop(['sales'], axis = 1)
y = train_data['next']
model = XGBRegressor(n_estimators=1000)
model.fit(X, y)
yhat = model.predict(test_data.drop(['next', 'Month', 'Year'], axis = 1))
test_data['pred next'] = yhat
print(mape(yhat, test_data['next'])

mar = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 3)]
apr = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 4)]
may = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 5)]

print(mape(mar['pred next'], mar['next']))
print(mape(apr['pred next'], apr['next']))
print(mape(may['pred next'], may['next']))

In [ ]:
####### LightBGM Forecasting #######

In [ ]:
df = pd.read_excel('Train_data.xlsx')
df[datetime(2020, 4, 1, 0, 0)] = (df[datetime(2020, 3, 1, 0, 0)] + df[datetime(2020, 5, 1, 0, 0)])/2
df = df.melt(id_vars=['Warehouse id', 'Region', 'SKU id'], var_name='date', value_name='sales')
df['SKU id'] = df['SKU id'].str.extract('(\d+)',expand = False).astype(int)
df.rename(columns={'SKU id':'store_id'}, inplace=True)
df = pd.concat([df,pd.get_dummies(df['Region'])], axis = 1)
df.drop('Region', axis = 1, inplace = True)
df['Year'] = df['date'].apply(lambda time: time.year)
df['Month'] = df['date'].apply(lambda time: time.month)
df.sort_values(['Warehouse id','store_id']).drop('Warehouse id', axis = 1)
df['isFest'] = df['date'].apply(lambda x: x.month == 3 or x.month == 5 or x.month == 11 or x.month == 12).astype(int)
df['isSum'] =  df['date'].apply(lambda x: x.month >= 4 and x.month<=7).astype(int)
df['isWin'] =  df['date'].apply(lambda x: x.month == 10 or x.month == 11 or x.month == 12 or x.month == 1 or x.month == 2).astype(int)
df = df.sort_values(['Warehouse id','store_id'])#.drop('Warehouse id', axis = 1)
df['next'] = df.groupby(['store_id', 'Warehouse id'])['sales'].shift(-1)
df['next'] = df['next'].fillna(df['next'].median())
# df['next'].fillna(df.groupby(['store_id', 'Warehouse id'])['sales'].median())
# df.head()
a = df.groupby(['store_id', 'Warehouse id'])['sales'].rolling(6).mean()
df['ra'] = a.fillna(a.mean()).reset_index(level=0, drop=True).reset_index(level=0, drop=True)
df['ewm'] = df.groupby(['store_id', 'Warehouse id'])['sales'].ewm(alpha = 0.2).mean().reset_index(level=0, drop=True).reset_index(level = 0, drop = True)
df.fillna(df.median(), inplace = True)

split_pt = '2020-10-01'
train_data = df[df['date'] <= split_pt]
test_data = df[df['date'] > split_pt]
# train_data

train_data.drop(['Warehouse id', 'date', 'store_id'], axis = 1, inplace = True)
test_data.drop(['Warehouse id', 'date', 'store_id'], axis = 1, inplace = True)

X = train_data.drop(['next', 'Month', 'Year'], axis = 1)
# X = train_data.drop(['sales'], axis = 1)
y = train_data['next']
model = lgbm.LGBMRegressor(n_estimators=1000)
model.fit(X, y)
yhat = model.predict(test_data.drop(['next', 'Month', 'Year'], axis = 1))
test_data['pred next'] = yhat
print(mape(yhat, test_data['next'])

mar = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 3)]
apr = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 4)]
may = test_data[(test_data['Year'] == 2021) & (test_data['Month'] == 5)]

print(mape(mar['pred next'], mar['next']))
print(mape(apr['pred next'], apr['next']))
print(mape(may['pred next'], may['next']))

In [ ]:
###### RandomForestRegressor ######

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_excel('Train_data.xlsx')
# data.head(6)
data_grouped_by_mean = data.groupby(by = 'Warehouse id').mean()
data_grouped_by_mean
data_melt = data.melt(id_vars=['Warehouse id','Region','SKU id'],var_name = 'month',value_name = 'sales')
# data.head()
# data_melt = data_melt.sort_values(['month','SKU id'])
data_melt.head(6)
data_melt['SKU id'] = data_melt['SKU id'].str.extract('(\d+)',expand = False).astype(int)
# data_melt['Warehouse id'] = data_melt['Warehouse id'].str.extract('(\d+)',expand = False).astype(int)
# data_melt = 
data_melt = data_melt.sort_values(['SKU id','Region','month'])
data_melt.head()
data_melt['sales next'] = data_melt.groupby(['SKU id','Region'])['sales'].shift(-1)
data_melt['sales prev'] = data_melt.groupby(['SKU id','Region'])['sales'].shift(1)
data_melt['sales diff'] = data_melt.groupby(['SKU id','Region'])['sales'].diff(1)
# data_melt[(data_melt['SKU id'] == 1) & (data_melt['Region'] == 'NORTH')].head(10)
data_melt.head(10)
data_melt['mean sales 4'] = data_melt.groupby('SKU id')['sales'].rolling(window = 4,center = True).mean().reset_index(level = 0,drop = True)  
data_melt['exp mean sales 4'] = data_melt.groupby('SKU id')['sales'].ewm(span=4).mean().reset_index(level = 0,drop = True)  
data_melt['year'] =data_melt['month'].apply(lambda x:x.year).astype(int)
data_melt.head()
data_melt['is Festival Month'] = data_melt['month'].apply(lambda x: x.month == 1 or x.month == 3 or x.month == 11 or x.month == 12 ).astype(int)
data_melt['is Summer'] = data_melt['month'].apply(lambda x: x.month == 5 or x.month == 6 or x.month == 7 or x.month == 8 ).astype(int)
data_melt['is Winter'] = data_melt['month'].apply(lambda x: x.month == 11 or x.month == 12 or x.month == 1 or x.month == 2 ).astype(int)
data_melt.head(10)
data_melt.fillna(data_melt.mean(),inplace = True)
# warehouse_encode = pd.get_dummies(data_melt['Warehouse id'], prefix = 'Warehouse')
# warehouse_encode.head()
region_encode = pd.get_dummies(data_melt['Region'],prefix = 'Region')
data_melt.drop(labels = ['Warehouse id'],axis =1 ,inplace = True)
encoded = pd.concat([region_encode,data_melt],axis = 1)
encoded.reset_index(drop = True,inplace = True)
encoded.head() 

split_point = '2021-01-01'
train = encoded[encoded['month'] < split_point].copy()
test = encoded[encoded['month'] >= split_point].copy()
# train.rest_index(inplace = True)
len(test)
# train.head()

train['month'] = train['month'].apply(lambda x:x.month).astype(int)
test['month'] = test['month'].apply(lambda x:x.month).astype(int)
# test['month'] = test['month'].apply(lambda x:x.month).astype(int)

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 100,random_state = 0)
Xtr = train.drop(['sales next','Region'],axis = 1)
Ytr = train['sales next']
# Xtr = train.drop(['sales','Region','sales diff'],axis = 1)
# Ytr = train['sales']
model.fit(Xtr,Ytr)

Ytrue = test['sales next']
Ypred = model.predict(test.drop(['sales next','Region'],axis = 1))
# Ytrue = test['sales']
# Ypred = model.predict(test.drop(['sales','Region','sales diff'],axis = 1))
print(mape(Ytrue,Ypred))

test['pred'] = Ypred
# plt.plot(test['sales next'])
# plt.plot(test['pred'])
test.reset_index(inplace = True)
test.sort_values(['SKU id'],inplace = True)
test1 = test[test['month'] == 1 ]
test2 = test[test['month'] == 2 ]
test3 = test[test['month'] == 3]
test4 = test[test['month'] == 4]
test1.reset_index(inplace = True)
test2.reset_index(inplace = True)
test3.reset_index(inplace = True)
test4.reset_index(inplace = True)

In [ ]:
####### DeepAR Pytorch-Forecasting #######

In [ ]:
!pip install pytorch-forecasting
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd

matplotlib.rcParams['figure.figsize'] = (10, 8)
matplotlib.rcParams['axes.grid'] = False
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
df = pd.read_excel('/content/Train_data.xlsx')
# df.drop(datetime(2020,4,1,0,0), axis=1, inplace=True)
df = df.melt(id_vars=['Warehouse id', 'Region', 'SKU id'], var_name='date', value_name='sales')
# df['SKU id'] = df['SKU id'].str.extract('(\d+)',expand = False).astype(int)
# df.rename(columns={'SKU id':'store_id'}, inplace=True)
# df.head()

# Feature Engineering
df['sales previous month'] = df.groupby(['Region', 'SKU id'])['sales'].shift(1)
df['diff sales 1'] = df.groupby(['Region', 'SKU id'])['sales'].diff(1)
df.sort_values(['Region', 'SKU id', 'date'], inplace=True)
df['rolling mean sales 4'] = df.groupby(['Warehouse id', 'SKU id'])['sales'].rolling(4).mean().reset_index(level=0, drop=True).reset_index(level=0, drop=True)
df['exp mean'] = df.groupby(['Warehouse id', 'SKU id'])['sales'].ewm(alpha=0.2).mean().reset_index(level=0, drop=True).reset_index(level=0, drop=True)

df['isHoli'] = df['date'].apply(lambda x: x.month == 3).astype(int)
df['isDiwali'] = df['date'].apply(lambda x: x.month == 10).astype(int)
df['isSankranti'] = df['date'].apply(lambda x: x.month == 1).astype(int)
df['isChristmas'] = df['date'].apply(lambda x: x.month == 12).astype(int)
df['isSummer'] = df['date'].apply(lambda x: x.month == 4 or x.month == 5 or x.month == 6).astype(int)
df['isWinter'] = df['date'].apply(lambda x: x.month == 11 or x.month == 12 or x.month == 1 or x.month == 2).astype(int)

df['isHoli'] = df['isHoli'].astype(str).astype("category")
df['isDiwali'] = df['isDiwali'].astype(str).astype("category")
df['isSankranti'] = df['isSankranti'].astype(str).astype("category")
df['isChristmas'] = df['isChristmas'].astype(str).astype("category")
df['isSummer'] = df.isSummer.astype(str).astype("category")
df['isWinter'] = df.isWinter.astype(str).astype("category")

df["month"] = df.date.dt.month.astype(str).astype("category")

df.sort_values(['Warehouse id', 'SKU id', 'date'], inplace=True)

df["time_idx"] = df["date"].dt.year * 12 + df["date"].dt.month
df["time_idx"] -= df["time_idx"].min()

df['Avg sales by SKU'] = df.groupby(["time_idx", "SKU id"], observed=True).sales.transform("mean")
df['Avg sales by Warehouse'] = df.groupby(["time_idx", "Warehouse id"], observed=True).sales.transform("mean")

df.reset_index(level=0, drop=True, inplace=True)
df.fillna(df.median(), inplace=True)

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
max_prediction_length = 6 # forecast 6 months
max_encoder_length = 38  # use 38 months of history
training_cutoff = df["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="sales",
    group_ids=["Warehouse id", "SKU id", 'Region'],
    min_encoder_length=0,  # allow predictions without history
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["Warehouse id", "SKU id", 'Region'],
    static_reals=[],
    time_varying_known_categoricals=['isHoli', 'isDiwali', 'isSankranti', 'isChristmas', 'isSummer', 'isWinter', "month"],
    # group of categorical variables can be treated as 
    # one variable
    variable_groups={"special_months": ['isHoli', 'isDiwali', 'isSankranti', 'isChristmas', 'isSummer', 'isWinter']},
    time_varying_known_reals=[
        "time_idx"
    ],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "sales",
        "sales previous month",
        "diff sales 1",
        "rolling mean sales 4",
        "Avg sales by SKU",
        "Avg sales by Warehouse"
    ],
    target_normalizer=GroupNormalizer(
        groups=["Warehouse id", "SKU id", "Region"]
    ),  # use softplus with beta=1.0 and normalize by group
    add_relative_time_idx=True,  # add as feature
    add_target_scales=True,  # add as feature
    add_encoder_length=True,  # add as feature
    allow_missing_timesteps=True
)
# create validation set (predict=True) which means to predict the
# last max_prediction_length points in time for each series
validation = TimeSeriesDataSet.from_dataset(
    training, df, predict=True, stop_randomization=True
)
# create dataloaders for model
batch_size = 128
train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=0
)
val_dataloader = validation.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=0
)

# imports for training
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
# import dataset, network to train and metric to optimize
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss

lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # log to tensorboard
# create trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")

trainer = pl.Trainer(
    max_epochs=30,
    gpus=[0],  # train on CPU, use gpus = [0] to run on GPU
    gradient_clip_val=0.1,
    limit_train_batches=30,  # running validation every 30 batches
    # fast_dev_run=True,  # comment in to quickly check for bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)
# initialise model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=21,  # biggest influence network size
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # log example every 10 batches
    reduce_on_plateau_patience=4,  # reduce learning automatically
)

res = trainer.tuner.lr_find(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

# from pytorch_forecasting.metrics import MAPE
import torch

# load the best model according to the validation loss (given that
# we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# calculate mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

print(predictions.shape)
print(mape(predictions[:,0].cpu().detach().numpy(), actuals[:,0].cpu().detach().numpy()))
print(mape(predictions[:,1].cpu().detach().numpy(), actuals[:,1].cpu().detach().numpy()))
print(mape(predictions[:,2].cpu().detach().numpy(), actuals[:,2].cpu().detach().numpy()))
print(mape(predictions[:,3].cpu().detach().numpy(), actuals[:,3].cpu().detach().numpy()))
print(mape(predictions[:,4].cpu().detach().numpy(), actuals[:,4].cpu().detach().numpy()))
print(mape(predictions[:,5].cpu().detach().numpy(), actuals[:,5].cpu().detach().numpy()))

In [ ]:
###### Prophet ######

In [ ]:
!java -version
!pip install pyspark
!pip install pyarrow==7.0.0
!pip install fbprophet

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from fbprophet import Prophet

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.master('local').getOrCreate()
df = pd.read_csv('train_data.csv')

df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['sales'] = df['sales'].astype(float)

sdf = spark.createDataFrame(df)
sdf.select(['store_id']).groupBy('store_id').agg({'store_id':'count'}).show()
sdf.createOrReplaceTempView("sales")
spark.sql("select store_id, count(*) from sales group by store_id order by store_id").show()
sql = 'SELECT store_id, date as ds, sum(sales) as y FROM sales GROUP BY store_id, ds ORDER BY store_id, ds'
spark.sql(sql).show()
sdf.rdd.getNumPartitions()
store_part = (spark.sql( sql ).repartition(spark.sparkContext.defaultParallelism, ['store_id'])).cache()
store_part.explain()

from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import current_date

result_schema =StructType([
  StructField('ds',TimestampType()),
  StructField('store_id',IntegerType()),
  StructField('y',DoubleType()),
  StructField('yhat',DoubleType()),
  StructField('yhat_upper',DoubleType()),
  StructField('yhat_lower',DoubleType())
  ])

@pandas_udf( result_schema, PandasUDFType.GROUPED_MAP )
def forecast_sales( store_pd ):

  model = Prophet(interval_width=0.95,seasonality_mode = 'multiplicative', yearly_seasonality=True)

  model.fit( store_pd )

  future_pd = model.make_future_dataframe(
    periods=5, 
    freq='m'
    )
  
  forecast_pd = model.predict( future_pd )  
  
  f_pd = forecast_pd[ ['ds','yhat', 'yhat_upper', 'yhat_lower'] ].set_index('ds')
  
  st_pd = store_pd[['ds','store_id','y']].set_index('ds')
  
  results_pd = f_pd.join( st_pd, how='left' )
  results_pd.reset_index(level=0, inplace=True)
  
  results_pd['store_id'] = store_pd['store_id'].iloc[0]

  return results_pd[ ['ds', 'store_id','y', 'yhat', 'yhat_upper', 'yhat_lower'] ]  